In [2]:
!pip install ta
import json
import pandas as pd
import requests
import datetime as dt
import os
import ta
import numpy as np
import yfinance as yf
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import statistics

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=07c5c567ad71e498701138b96599db2fd583672f6debf61914f4a26b0296ca20
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta
Mounted at /content/drive


In [4]:
def get_exchange_data(key, exchange='NYSE'):
    endpoint = f'https://eodhistoricaldata.com/api/exchange-symbol-list/'
    endpoint += f'{exchange}?api_token={key}&fmt=json'
    call = requests.get(endpoint).text
    exchange_data = pd.DataFrame(json.loads(call))
    return exchange_data

def get_security_type(exchange_data, type = 'Common Stock'):
    symbols = exchange_data[exchange_data.Type == type]
    return symbols.Code.tolist()


key = open('drive/MyDrive/api_token.txt').read()
x = get_security_type(get_exchange_data(key))

In [8]:

v_list = []
macd_list = []
rsi_list = []
ema_list = []
close_20 = []




for ticker in x[:20]:
    for year in ['5y']:
        t = yf.Ticker(ticker)
        # period = “1d”, “5d”, “1mo”, “3mo”, “6mo”, “1y”, “2y”, “5y”, “10y”, “ytd”, “max”
        # interval = “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”
        df = t.history(period = year)
        # df = t.history(start = '2020-03-18', end = '2022-03-20', interval="1d")
        df['long returns'] = np.log(df['Close']/df['Close'].shift())
        volatility = df['long returns'].std()*252**.5
        str_vol = str(round(volatility, 4)*100)
        df['Volatility'] = str_vol
        df['ema20'] = df['Close'].ewm(span=20, adjust=False).mean()
        df['ema50'] = df['Close'].ewm(span=50, adjust=False).mean()
        df['ema200'] = df['Close'].ewm(span=200, adjust=False).mean()
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['ema9'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['Ticker'] = ticker
        df['rsi'] = ta.momentum.rsi(df.Close, window=14)
        df['20-50'] = df['ema20'] - df['ema50']
        df['50-200'] = df['ema50'] - df['ema200']
        df['20-200'] = df['ema20'] - df['ema200']
        df['close-20'] = df['Close'] - df['ema20']
        df['macd()'] = df['MACD'] - df['ema9']
        df_2 = df.loc[:, ['Ticker','Close','20-200','macd()','rsi']]
        # try:
        #   df = df.drop(['High','Low','Volume','Dividends','Stock Splits'], axis=1)
        # except:
        #   KeyError
        df = df.loc[:, ['Ticker','Open','Close','ema20','ema50','ema200','MACD','ema9','20-50','50-200','20-200','close-20','macd()','rsi']]
        # display(df.tail(7))





       ## Plots
        # fig, ax = plt.subplots()
        # # df['Close'].plot(ax=ax)
        # ax.set_title(ticker + '-' + year + ' Close Price')
        # fig.set_size_inches(18, 10, forward=True)
        # fig, ax = plt.subplots()
        # df[['ema20','ema50','ema200','close-20']].plot(ax=ax)
        # ax.set_title(ticker + '-' + year + ' EMA')
        # fig.set_size_inches(18, 10, forward=True)
        # print()
        # fig, ax = plt.subplots()
        # df[['MACD','ema9']].plot(ax=ax)
        # ax.set_title(ticker + '-' + year + ' MACD(12,26,9)')
        # fig.set_size_inches(18, 10, forward=True)
        # print()
        # fig, ax = plt.subplots()
        # df['rsi'].plot(ax=ax)
        # fig.set_size_inches(18, 10, forward=True)
        # ax.set_title(ticker + '-' + year + ' RSI')
        # fig, ax = plt.subplots()
        # df[['20-50']].plot(ax=ax)
        # df[['50-200']].plot(ax=ax)
        # df[['20-200']].plot(ax=ax)
        # df[['close-20']].plot(ax=ax)
        # df['Close'].plot(ax=ax)
        # fig.set_size_inches(18, 10, forward=True)
        # ax.set_title(ticker + '-' + year + ' Close vs EMA')


##EMA and MACD

In [25]:
datetime.now()

datetime.datetime(2024, 3, 29, 16, 36, 21, 231135)

In [79]:
from datetime import datetime, timedelta, date

today = datetime(2024,3,28) # TODO: us holidays
asof_date = today - timedelta(1)

In [43]:
df.reset_index()
df.Date

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,long returns,Volatility,...,ema200,MACD,ema9,Ticker,rsi,20-50,50-200,20-200,close-20,macd()
0,2022-03-21 00:00:00-04:00,9.935294,11.000000,8.235294,9.264706,123760,0.0,0.0,NaN,129.02,...,9.264706,0.000000,0.000000,AAMC,NaN,0.000000,0.000000,0.000000,0.000000,0.000000
1,2022-03-22 00:00:00-04:00,8.655882,10.247059,8.352941,8.647059,143820,0.0,0.0,-0.068993,129.02,...,9.258560,-0.049271,-0.009854,AAMC,NaN,-0.034602,-0.018076,-0.052678,-0.558823,-0.039417
2,2022-03-23 00:00:00-04:00,8.764706,9.352941,8.147059,8.264706,102170,0.0,0.0,-0.045225,129.02,...,9.248671,-0.117813,-0.031446,AAMC,NaN,-0.085972,-0.046452,-0.132424,-0.851541,-0.086367
3,2022-03-24 00:00:00-04:00,8.326471,8.558824,8.294118,8.394118,12580,0.0,0.0,0.015537,129.02,...,9.240168,-0.159848,-0.057126,AAMC,NaN,-0.123056,-0.069640,-0.192696,-0.653354,-0.102722
4,2022-03-25 00:00:00-04:00,8.352941,9.250000,8.352941,8.958824,56610,0.0,0.0,0.065108,129.02,...,9.237368,-0.145912,-0.074884,AAMC,NaN,-0.123197,-0.075142,-0.198339,-0.080205,-0.071029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2024-03-22 00:00:00-04:00,3.600000,3.600000,3.220000,3.220000,3900,0.0,0.0,-0.097566,129.02,...,9.712254,-0.230993,-0.215276,AAMC,36.777586,-0.341515,-5.610634,-5.952149,-0.540105,-0.015717
505,2024-03-25 00:00:00-04:00,3.480000,3.480000,3.480000,3.480000,1400,0.0,0.0,0.077651,129.02,...,9.650242,-0.224891,-0.217199,AAMC,42.893165,-0.343815,-5.572998,-5.916813,-0.253428,-0.007691
506,2024-03-26 00:00:00-04:00,3.480000,3.480000,3.480000,3.480000,0,0.0,0.0,0.000000,129.02,...,9.588846,-0.217547,-0.217269,AAMC,42.893165,-0.344530,-5.535024,-5.879554,-0.229292,-0.000278
507,2024-03-27 00:00:00-04:00,3.450000,3.450000,3.450000,3.450000,400,0.0,0.0,-0.008658,129.02,...,9.527763,-0.211707,-0.216157,AAMC,42.345032,-0.345545,-5.497621,-5.843165,-0.234598,0.004449


In [106]:
df_2_list = []

for ticker in x[:20]:
    for year in ['5y']:
        t = yf.Ticker(ticker)
        # period = “1d”, “5d”, “1mo”, “3mo”, “6mo”, “1y”, “2y”, “5y”, “10y”, “ytd”, “max”
        # interval = “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”
        df = t.history(period = year)
        # df = t.history(start = '2022-06-10', end = '2022-06-15', interval="1d")
        df['long returns'] = np.log(df['Close']/df['Close'].shift())
        volatility = df['long returns'].std()*252**.5
        str_vol = str(round(volatility, 4)*100)
        df['Volatility'] = str_vol
        df['ema20'] = df['Close'].ewm(span=20, adjust=False).mean()
        df['ema50'] = df['Close'].ewm(span=50, adjust=False).mean()
        df['ema200'] = df['Close'].ewm(span=200, adjust=False).mean()
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['ema9'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['Ticker'] = ticker
        df['rsi'] = ta.momentum.rsi(df.Close, window=14)
        df['20-50'] = df['ema20'] - df['ema50']
        df['50-200'] = df['ema50'] - df['ema200']
        df['20-200'] = df['ema20'] - df['ema200']
        df['close-20'] = df['Close'] - df['ema20']
        df['macd()'] = df['MACD'] - df['ema9']
        df_2 = df.loc[:, ['Ticker','Volatility','Close','20-200','close-20', 'macd()','rsi']]
        df_2['Year'] = year
        df_2_list.append(df_2.tail(1).reset_index())
        # display(df_2.sort_values(by = '20-200', ascending=False).head())


In [107]:
df_final = pd.concat(df_2_list, axis=0, ignore_index=True)
df_final['Date'] = pd.to_datetime(df_final['Date']).dt.date
df_final[df_final.Date < date.today()]
df_final[:10]

,Date,Ticker,Volatility,Close,20-200,close-20,macd(),rsi,Year
0,2024-03-28,A,29.549999999999997,145.509995,14.492980,0.770595,-0.606711,54.929346,5y
1,2024-03-28,AA,62.57,33.790001,-0.210179,3.124267,0.334884,68.981785,5y
2,2024-03-28,AACT,2.13,10.570000,0.181865,0.014829,-0.001258,60.379848,5y
3,2024-03-28,AACT-WT,nan,0.185000,0.000000,0.000000,0.000000,NaN,5y
4,2024-03-28,AAMC,129.02,3.260000,-5.821237,-0.384160,-0.002976,38.950152,5y
5,2024-03-28,AAN,59.68,7.500000,-2.401390,-0.034354,0.156557,44.667521,5y
6,2024-03-28,AAP,41.07,85.089996,5.182788,5.313755,0.094678,74.046993,5y
7,2024-03-28,AAT,38.57,21.910000,0.424106,0.576095,0.095070,58.786020,5y
8,2024-03-28,AAU,77.86999999999999,0.140000,-0.007515,-0.000005,0.000011,48.358243,5y
9,2024-03-28,AB,39.24,34.740002,2.316140,0.861396,0.052329,60.955745,5y


In [9]:
for ticker in x[:20]:
    for year in ['5y']:
        t = yf.Ticker(ticker)
        # period = “1d”, “5d”, “1mo”, “3mo”, “6mo”, “1y”, “2y”, “5y”, “10y”, “ytd”, “max”
        # interval = “1m”, “2m”, “5m”, “15m”, “30m”, “60m”, “90m”, “1h”, “1d”, “5d”, “1wk”, “1mo”, “3mo”
        df = t.history(period = year)
        # df = t.history(start = '2022-06-10', end = '2022-06-15', interval="1d")
        df['long returns'] = np.log(df['Close']/df['Close'].shift())
        volatility = df['long returns'].std()*252**.5
        str_vol = str(round(volatility, 4)*100)
        df['Volatility'] = str_vol
        df['ema20'] = df['Close'].ewm(span=20, adjust=False).mean()
        df['ema50'] = df['Close'].ewm(span=50, adjust=False).mean()
        df['ema200'] = df['Close'].ewm(span=200, adjust=False).mean()
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['ema9'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['Ticker'] = ticker
        df['rsi'] = ta.momentum.rsi(df.Close, window=14)
        df['20-50'] = df['ema20'] - df['ema50']
        df['50-200'] = df['ema50'] - df['ema200']
        df['20-200'] = df['ema20'] - df['ema200']
        df['close-20'] = df['Close'] - df['ema20']
        df['macd()'] = df['MACD'] - df['ema9']
        df_2 = df.loc[:, ['Ticker','Close','20-200','close-20', 'macd()','rsi']]
        # display(df_2.sort_values(by = '20-200', ascending=False).head())





##Volatility

        if float(str_vol) > 20:
          print(ticker +"'s volatility is : " + str(str_vol) + "%. Watch it.")
          vo = '1'
          v_list.append(vo)
        else:
          print(ticker +"'s volatility is less than 50%. No signal.")
          vo = '0'
          v_list.append(vo)


##MACD

        avg_sp = df_2['macd()'].tail(100).mean()
        std = df_2['macd()'].tail(100).std()
        current = df_2.tail(1).iloc[0,3]
        current_sp = current * 2 * std
        rsi = df_2.tail(1).iloc[0,5]
        if current_sp > avg_sp:
            # print('current spread + 2 std is: ')
            # print(current_sp)
            # print('average spread is:')
            # print(avg_sp )
            # print()
            print('Current macd is greater than average macd. Watch it.')
            ma = '1'
            macd_list.append(ma)
        else:
            print('Current macd is less than average macd. No signal.')
            ma = '0'
            macd_list.append(ma)

##RSI

        if rsi < 30:
            print("rsi's value is "+ str(rsi)+', which is less than 30. Watch it.')
            rs = '1'
            rsi_list.append(rs)
        elif rsi > 70:
            print("rsi's value is")
            print(rsi)
            print('It is greater than 70. Watch it.')
            rs = '1'
            rsi_list.append(rs)
        else:
            print(ticker + "rsi's value is "+ str(rsi)+', which is between 30 and 70. No signal.')
            rs = '0'
            rsi_list.append(rs)




##EMA20-200

        current_ema = df_2.tail(1).iloc[0,2]
        ema_max = df_2['20-200'].sort_values(ascending=False).head(100).mean()
        ema_min = df_2['20-200'].sort_values(ascending=True).head(100).mean()

        if current_ema > ema_max:
            print('Current ema20-200 is greater than ema max. Watch it.')
            em = '1'
            ema_list.append(em)
        elif current_ema < ema_min:
            print('Current ema20-200 is less than ema minimum. Watch it.')
            em = '1'
            ema_list.append(em)
        else:
            print('Current ema20-200 is between max and min. No signal.')
            em = '0'
            ema_list.append(em)


##Close-EMA20

        current_close_20 = df_2.tail(1).iloc[0,3]
        close_20_max = df_2['close-20'].sort_values(ascending=False).head(100).mean()
        close_20_min = df_2['close-20'].sort_values(ascending=True).head(100).mean()

        if current_close_20 > close_20_max:
           print('Current close price - ema 20 is greater than max. Watch it.')
           clo = '1'
           close_20.append(clo)
        elif current_close_20 < close_20_min:
           print('Current close price - ema 20 is less than min. Watch it.')
           clo = '1'
           close_20.append(clo)
        else:
           print('Current close price - ema 20  is between max and min. No signal.')
           clo = '0'
           close_20.append(clo)
           print()





A's volatility is less than 50%. No signal.
Current macd is greater than average macd. Watch it.
Arsi's value is 54.929346359633534, which is between 30 and 70. No signal.
Current ema20-200 is between max and min. No signal.
Current close price - ema 20  is between max and min. No signal.

AA's volatility is : 62.57%. Watch it.
Current macd is greater than average macd. Watch it.
AArsi's value is 68.98178528337985, which is between 30 and 70. No signal.
Current ema20-200 is between max and min. No signal.
Current close price - ema 20  is between max and min. No signal.

AACT's volatility is less than 50%. No signal.
Current macd is less than average macd. No signal.
AACTrsi's value is 60.379847867152215, which is between 30 and 70. No signal.
Current ema20-200 is greater than ema max. Watch it.
Current close price - ema 20  is between max and min. No signal.

AACT-WT's volatility is less than 50%. No signal.
Current macd is less than average macd. No signal.
AACT-WTrsi's value is nan, 